# Prepatation

## Instaling Libararies

In [2]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from scipy.optimize import minimize
import pytz
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
import seaborn as sns
# from sklearn.model_selection import TimeSeriesSplit
import statsmodels.tsa.api as smt
from copy import deepcopy
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic
import warnings
warnings.filterwarnings("ignore")
import statsmodels.tools.eval_measures
import scipy 
from scipy import stats
import time
from matplotlib.dates import DateFormatter
from matplotlib.dates import YearLocator, DateFormatter
from statsmodels.tsa.api import VAR

In [3]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

## Inporting data

In [5]:
df_base = pd.read_csv('../23_10_27_week-03/stock_da.csv')

In [6]:
df_base

,Unnamed: 0,TYO,F,GM,HMC,STLA,TOYOF
0,2013-10-01,24.750,17.19,35.91,38.22,8.06,63.8800
1,2013-10-02,24.629,17.21,35.94,38.28,8.06,63.2400
2,2013-10-03,24.580,16.95,35.29,38.07,8.06,63.3501
3,2013-10-04,24.820,17.09,35.70,38.37,8.35,63.9200
4,2013-10-07,24.630,16.81,35.18,37.91,8.60,63.9200
...,...,...,...,...,...,...,...
2512,2023-09-25,15.210,12.58,33.06,35.21,19.24,18.4750
2513,2023-09-26,15.280,12.43,32.26,34.73,18.84,18.0450
2514,2023-09-27,15.510,12.39,32.35,34.99,18.81,18.1900
2515,2023-09-28,15.390,12.56,33.16,34.84,19.27,18.2650


# データの前処理